In [82]:
import os
import sys
import time
import random
import warnings
import collections
from dateutil.relativedelta import relativedelta
from datetime import datetime
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Script to generate supplemental file for claim submission
Currently configured for UHC, other plans may be signifigantly different

In [83]:
# configuration
use_cache = False
seed = random.randint(0, 100)
from_date = '2022-03-23'

print(f'Seed: {seed}')

Seed: 74


### Pull Data

In [84]:
query = f"""
SELECT * FROM perm.supp_file_uhc_20230608;
"""

In [85]:
df = cb_utils.sql_query_to_df(query, use_cache=use_cache)

Pulling query from db


In [86]:
df.shape # 4609 -> 6954

(13995, 23)

In [87]:
df.mbi.nunique(),  df.member_id.nunique()

(10432, 10447)

In [88]:
df.head()

,elation_bill_id,patient_first_name,patient_last_name,patient_dob,mbi,member_id,gender,date_of_service,provider_first_name,provider_last_name,provider_email,provider_npi,medicare_specialty_code_int,tax_id,place_of_service,ra_code,line1,line2,city,state,postal_code,procedure_code,icds
0,493387270586524,Edith,Schlayer,1950-03-08,7GJ5T84QD50,106739268,F,2022-07-25,Jessica,Flippo,jessica.flippo@carebridgehealth.com,1194221317,50,84-2590508,10,A,601 E BARTON RIDGE RD,Apt 102,GREENEVILLE,TN,37745,99212,"[E66.01, I50.9, E26.1, E11.42, E11.42, E11.51,..."
1,614232323915932,Isaiah,Bear,1995-10-30,6H86JY1DW54,114294827,M,2023-06-05,Susana,Sanchez,Susana.Sanchez@carebridgehealth.com,1720518137,50,84-2590508,10,A,584 DELORES DR,None,Dandridge,TN,37725,99214,"[H54.7, Z68.1, H54.7, H54.7, H91.90, I10, R56...."
2,556769413890204,Thomas,Hayes,1982-10-25,6AH0C76FQ17,116891229,M,2023-01-17,Jessica,Flippo,jessica.flippo@carebridgehealth.com,1194221317,50,84-2590508,10,A,2242 DRY VALLEY RD,None,THORN HILL,TN,37881,99214,"[S24.153S, G82.22, M62.838, N31.9]"
3,559953381752988,Betty,Ray,1937-03-16,7QY0U85FJ05,106731291,F,2023-01-25,Lacey,Bolden,lacey.bolden@carebridgehealth.com,1740410703,50,84-2590508,10,A,2114 DUNCAN AVE,None,CHATTANOOGA,TN,37404,99214,"[E11.22, I13.0, I50.9, N18.31, J44.9, J96.11, ..."
4,491633387700380,Joe,Hammonds,1954-08-10,1DV6GR3MC46,106750874,M,2022-07-20,Lacey,Bolden,lacey.bolden@carebridgehealth.com,1740410703,50,84-2590508,10,A,1201 BOYNTON DR APT 805,None,CHATTANOOGA,TN,37402,99214,"[G40.919, I69.354, Z79.01, I10, E11.69, E11.69..."


In [89]:
# dupe check
assert df.elation_bill_id.nunique() == df.shape[0]

In [90]:
# icd check
assert df.loc[df.icds.isna()].shape[0] == 0

In [91]:
# if more than 40 we need to duplicate the row and add the additional icds on the dupe row
# currently not coded up because the situation doesn't exist
assert df.icds.apply(lambda x: x if x is None else len(set(x))).max() <= 40

#### Pad dx list col to 40

In [92]:
def pad_dx_col(icds):
    if icds is None:
        return [None] * 40
    
    icds = [i.replace('.', '') for i in set(icds)]
    
    if len(icds) < 40:
        return icds + [None] * (40 - len(icds))
    return icds
    
    
df.icds = df.icds.apply(pad_dx_col)

### Format dates

In [93]:
df.date_of_service = pd.to_datetime(df.date_of_service).dt.strftime('%m/%d/%Y')
df.patient_dob = pd.to_datetime(df.patient_dob).dt.strftime('%m/%d/%Y')

### Pad CMS IDs

In [94]:
df.place_of_service = df.place_of_service.str.pad(width=2, side='left', fillchar='0')
df.medicare_specialty_code_int = df.medicare_specialty_code_int.str.pad(width=2, side='left', fillchar='0')

AttributeError: Can only use .str accessor with string values!

### Build rows

In [ ]:
def build_file_row(i, row):
    return [
        'DTL', # seg type
        row.elation_bill_id, # ref #
        row.patient_last_name,
        row.patient_first_name,
        None, # MI
        row.patient_dob,
        row.member_id if row.mbi is None else None, # member id
        None, # 'Retrieval NPI (Internal Use)',
        row.gender, # 'GENDER',
        None, # 'STATE CODE',
        row.mbi, # 'MBI-  need either member ID or MBI',
        row.date_of_service, # 'FDOS',
        row.date_of_service, # 'TDOS',
        None, # 'BILL TYPE- Institutional Only',
        None, # 'NU Indicator (Internal Use)',
        None, # 'PROV ID',
        row.provider_npi, # 'NPI',
        None, # 'PROV TYPE',
        None, # 'FACILITY NM- required for Institutional',
        row.provider_last_name, # 'PROV LAST NAME- Required for Professional',
        row.provider_first_name, # 'PROV FIRST NAME- required for Professional',
        f'{row.medicare_specialty_code_int}', # 'CMS SPECIALTY- Required for Professional',
        row.tax_id, # 'TAX ID',
        row.procedure_code, # 'CPT- Professional and Hospital Outpatient only',
        None, # 'REV CODE - Required for Institutional',
        None, # 'SERVICE FDOS',
        None, # 'SERVICE TDOS',
        row.place_of_service, # 'POS- Professional only',
        '0', # 'ICD INDIC',
        row.ra_code, # 'RA Code- Required for Professional',
        None, # 'Chart Barcode (Internal Use)',
        None, # 'Chart Enc Key (Internal Use)',
        None, # 'Chart DX Key  (Internal Use)',
        None, # 'Contract ID (Tufts use only)',
        row.line1, # 'Mem Street Address ',
        row.line2, # 'Mem Address 2',
        row.city, # 'Mem City',
        row.state, # 'Mem State',
        row.postal_code, # 'Mem Zip Code',
        None, # 'CLAIMID/PCN', -- This would link it to an existing claim and make it a resubmit
    ] + row.icds

rows = []
for i, row in df.iterrows():
    rows.append(build_file_row(i, row))

### Add header and footer rows

In [ ]:
n_rows, n_columns = len(rows), len(rows[0])
# 100k limit per file
assert n_rows <= 100000

training_partner_id = '00795'
file_name = f'{training_partner_id}_UHC_ASMP_DIRECT_{datetime.now().strftime("%Y%m%d%H%M%S")}.txt'

header_vals = [
    'HDR',
    '8.9',
    training_partner_id,
    file_name,
    'UHC',
    'DIRECT',
    'P',
    'Y'
]
header_vals = header_vals + [None] * (n_columns - len(header_vals))
footer_vals = [
    'TRL',
    training_partner_id,
    n_rows
]
footer_vals = footer_vals + [None] * (n_columns - len(footer_vals))

In [ ]:
rows.insert(0, header_vals)
rows.append(footer_vals)

### Build File

In [95]:
dx_cols = [f'DX_{i}' for i in range(40)]
columns = [
    'SEG TYPE',
    'REF #',
    'LAST NAME',
    'FIRST NAME',
    'MI',
    'DOB',
    'MEMBER ID- need either member ID or MBI',
    'Retrieval NPI (Internal Use)',
    'GENDER',
    'STATE CODE',
    'MBI-  need either member ID or MBI',
    'FDOS',
    'TDOS',
    'BILL TYPE- Institutional Only',
    'NU Indicator (Internal Use)',
    'PROV ID',
    'NPI',
    'PROV TYPE',
    'FACILITY NM- required for Institutional',
    'PROV LAST NAME- Required for Professional',
    'PROV FIRST NAME- required for Professional',
    'CMS SPECIALTY- Required for Professional',
    'TAX ID',
    'CPT- Professional and Hospital Outpatient only',
    'REV CODE - Required for Institutional',
    'SERVICE FDOS',
    'SERVICE TDOS',
    'POS- Professional only',
    'ICD INDIC',
    'RA Code- Required for Professional',
    'Chart Barcode (Internal Use)',
    'Chart Enc Key (Internal Use)',
    'Chart DX Key  (Internal Use)',
    'Contract ID (Tufts use only)',
    'Mem Street Address ',
    'Mem Address 2',
    'Mem City',
    'Mem State',
    'Mem Zip Code',
    'CLAIMID/PCN',
] + dx_cols

In [96]:
file_df = pd.DataFrame(rows, columns=columns)

In [97]:
file_df.head(20)

,SEG TYPE,REF #,LAST NAME,FIRST NAME,MI,DOB,MEMBER ID- need either member ID or MBI,Retrieval NPI (Internal Use),GENDER,STATE CODE,MBI- need either member ID or MBI,FDOS,TDOS,BILL TYPE- Institutional Only,NU Indicator (Internal Use),PROV ID,NPI,PROV TYPE,FACILITY NM- required for Institutional,PROV LAST NAME- Required for Professional,PROV FIRST NAME- required for Professional,CMS SPECIALTY- Required for Professional,TAX ID,CPT- Professional and Hospital Outpatient only,REV CODE - Required for Institutional,SERVICE FDOS,SERVICE TDOS,POS- Professional only,ICD INDIC,RA Code- Required for Professional,Chart Barcode (Internal Use),Chart Enc Key (Internal Use),Chart DX Key (Internal Use),Contract ID (Tufts use only),Mem Street Address,Mem Address 2,Mem City,Mem State,Mem Zip Code,CLAIMID/PCN,DX_0,DX_1,DX_2,DX_3,DX_4,DX_5,DX_6,DX_7,DX_8,DX_9,DX_10,DX_11,DX_12,DX_13,DX_14,DX_15,DX_16,DX_17,DX_18,DX_19,DX_20,DX_21,DX_22,DX_23,DX_24,DX_25,DX_26,DX_27,DX_28,DX_29,DX_30,DX_31,DX_32,DX_33,DX_34,DX_35,DX_36,DX_37,DX_38,DX_39
0,HDR,8.9,00795,00795_UHC_ASMP_DIRECT_20230615160853.txt,UHC,DIRECT,P,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,DTL,493387270586524,Schlayer,Edith,None,03/08/1950,None,None,F,None,7GJ5T84QD50,07/25/2022,07/25/2022,None,None,None,1194221317,None,None,Flippo,Jessica,50,84-2590508,99212,None,None,None,10,0,A,None,None,None,None,601 E BARTON RIDGE RD,Apt 102,GREENEVILLE,TN,37745,None,F0390,E261,F419,E6601,I509,F330,E1142,G4730,E1151,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,DTL,614232323915932,Bear,Isaiah,None,10/30/1995,None,None,M,None,6H86JY1DW54,06/05/2023,06/05/2023,None,None,None,1720518137,None,None,Sanchez,Susana,50,84-2590508,99214,None,None,None,10,0,A,None,None,None,None,584 DELORES DR,None,Dandridge,TN,37725,None,Z931,Z681,R569,I10,F72,H9190,G809,Q8789,E441,H547,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,DTL,556769413890204,Hayes,Thomas,None,10/25/1982,None,None,M,None,6AH0C76FQ17,01/17/2023,01/17/2023,None,None,None,1194221317,None,None,Flippo,Jessica,50,84-2590508,99214,None,None,None,10,0,A,None,None,None,None,2242 DRY VALLEY RD,None,THORN HILL,TN,37881,None,S24153S,M62838,G8222,N319,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,DTL,559953381752988,Ray,Betty,None,03/16/1937,None,None,F,None,7QY0U85FJ05,01/25/2023,01/25/2023,None,None,None,1740410703,None,None,Bolden,Lacey,50,84-2590508,99214,None,None,None,10,0,A,None,None,None,None,2114 DUNCAN AVE,None,CHATTANOOGA,TN,37404,None,F0390,K219,E6601,I509,F1120,J449,E785,Z7409,J9611,I69354,I130,N1831,E1142,E1122,Z6839,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,DTL,491633387700380,Hammonds,Joe,None,08/10/1954,None,None,M,None,1DV6GR3MC46,07/20/2022,07/20/2022,None,None,None,1740410703,None,None,Bolden,Lacey,50,84-2590508,99214,None,None,None,10,0,A,None,None,None,None,1201 BOYNTON DR APT 805,None,CHATTANOOGA,TN,37402,None,E785,F5104,I69354,Z7901,E1169,I10,R531,G40919,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,DTL,572461896564892,Hammonds,Joe,None,08/10/1954,None,None,M,None,1DV6GR3MC46,02/27/2023,02/27/2023,None,None,None,1194221317,None,None,Flippo,Jessica,50,84-2590508,99214,None,None,None,10,

In [19]:
file_df.to_csv(f'../data/{file_name}', sep='|', index=False, header=False)

In [20]:
file_name

'00795_UHC_ASMP_DIRECT_20230608102118.txt'

### response

In [ ]:
import re

cols = [re.sub('[^0-9a-zA-Z]+', '_', c.lower()) for c in columns+['err_id', 'error']]
df = pd.read_csv('/Users/bp/workspace/cb/data/00795_UHC_ASMP_DIRECT_20220328162146_RESUB1.txt',
                 sep='|', header=None, skipfooter=1, skiprows=1, names=cols, engine='python'
                )
df.head()


In [ ]:
df.error.value_counts()

In [ ]:
# HDR|8.9|00795|00795_UHC_ASMP_DIRECT_20220328162146_RESUB1.txt|UHC|DIRECT|P|Y

In [ ]:
df.loc[df.error == 'err18-Blank MemberID; err40-Member not found in plan membership;']

In [1]:
df.to_csv('/Users/bp/Downloads/uhc_supp_file_errors_20230615.csv', index=False)

NameError: name 'df' is not defined